In [2]:
# credit : https://github.com/yunjey/pytorch-tutorial/tree/master/tutorials/02-intermediate

# MNIST dataset 
The MNIST database contains 60,000 training images and 10,000 testing images

<img src="mnist.png" width="250">

### Image size : 1x28x28 , __#__ of classes : 10


# torch.nn.Conv2d

<img src="conv2d.png" width="650">

### simple example 1 (in_channels=1,out_channels=3)

In [3]:
img = torch.rand(1,1,5,5) # batchsize x channels x height x width    
myconv = nn.Conv2d(1,3,kernel_size=3)
out = myconv(img)
print(out.size())

torch.Size([1, 3, 3, 3])


<img src="conv_ex1.png" width="650">

### weight and bias size

In [4]:
print(myconv.weight.size())
print(myconv.bias.size())

torch.Size([3, 1, 3, 3])
torch.Size([3])


### simple example 2 (in_channels=2,out_channels=3)

In [5]:
featuremap = torch.rand(1,2,5,5) # batchsize x channels x height x width 
myconv = nn.Conv2d(2,3,kernel_size=3)
out = myconv(featuremap)
print(out.size())

torch.Size([1, 3, 3, 3])


<img src="conv_ex2.png" width="650">

### weight and bias size 

In [6]:
print(myconv.weight.size())
print(myconv.bias.size())

torch.Size([3, 2, 3, 3])
torch.Size([3])


### simple example 3 (padding)
P = (F-1)/2
where P is number of zero padding, F is the filter size(odd number) and the number of stride is 1. 

In [7]:
featuremap = torch.rand(1,2,5,5) # batchsize x channels x height x width    
myconv = nn.Conv2d(2,2,kernel_size=5,padding=2) 
out = myconv(featuremap)
print(out.size())

torch.Size([1, 2, 5, 5])


### simple example 4 (stride)

In [ ]:
featuremap = torch.rand(1,2,10,10) # batchsize x channels x height x width    
myconv = nn.Conv2d(2,2,kernel_size=3,padding=1,stride=2) 
out = myconv(featuremap)
print(out.size())

<img src="conv_ex4.png" width="650">

### simple example 5 (pooling)

In [ ]:
featuremap = torch.rand(1,2,10,10) # batchsize x channels x height x width    
mypool = nn.MaxPool2d(kernel_size=2, stride=2)
out = mypool(featuremap)
print(out.size())

<img src="conv_ex5.png" width="650">

# MAIN code

In [1]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms


# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Hyper parameters
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='../../data/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data/',
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

# Convolutional neural network (two convolutional layers)
# 1x28x28
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7*7*32, num_classes)
        
    def forward(self, x):
        1x28x28
        out = self.layer1(x)
        16x14x14
        out = self.layer2(out)
        32x7x7
        out = out.reshape(out.size(0), -1)
        batchsize x(32x7x7)
        out = self.fc(out)
        batchsize x 10
        return out

model = ConvNet(num_classes).to(device)



Processing...
Done!


In [ ]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
# torch.save(model.state_dict(), 'model.ckpt')